In [ ]:
import pandas as pd
import numpy as np
import operator
import pickle
import networkx as nx
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
from matplotlib import pyplot as plt
from nn_utils import MatrixFactorization
from itertools import zip_longest,combinations
import math
from datetime import datetime
from collections import defaultdict
import copy
import scipy.sparse as sp
from recommendation_utils import get_ib_recommendations, get_pop_recommendations,get_mf_recommendations,get_ub_recommendations, get_nn_recommendations, get_random_recommendations,get_simrank_recommendations
import gc
import pycmf

We define the experiment parameters

In [ ]:
dataset = 'mimic'  #valid options are decagon,medical,movielens,mimic
seed = 0 # By changing this value, we get differenet random splits for train, test sets
train_test_split = 0.8 # This value shows the % of the train and data set
min_items = 15
top_n = 5 # This is used for creating precision - recall charts, 10 for mimic
neighborhood = 80 # The neighborhood to use for using UBCF
similarity_coefficient = 0.5
train_batch =512 # Used for training the NN
methods = ['Pathsim','IBCF','MF','Pathsim_ddi','IBCF_ddi','MF_ddi']
ddi_datasets = ['mimic']

We load the dataset 

In [ ]:
df = pd.read_csv('mimic.csv')
columns = ['patient','drug']
df = df[columns]
columns = ['user','drug']
df_ddi = pd.read_csv('mimic_ddi.csv')
df_ddi = df_ddi[['Drug1','Drug2']]    
df.columns = columns
df.head()


In [ ]:

temp_df = df.groupby('user')['drug'].unique()
medicines=[]
counter =0
counter_temp= []
for i in range(0, len(temp_df)):
    if len(temp_df.iloc[i]) > 15:
        counter_temp.append(counter)
        user_medicine = f"{temp_df.iloc[i][0]}"
        for drug in range(1, len(temp_df.iloc[i])):
            user_medicine += f' {temp_df.iloc[i][drug]}'
        medicines.append(user_medicine)
    counter+=1
all_users = list(df['user'].unique())
users_to_keep = []
for user in all_users:
    if len(temp_df.loc[user])>min_items:
        users_to_keep.append(user)
train_df = df[df.user.isin(users_to_keep)]
train_df =  train_df.drop_duplicates()
train_df = train_df.sort_values(by='user')
train_df = train_df.reset_index()
train_df = train_df.drop('index',axis=1)
# records=dill.load(open('records_final_for_my.pkl','rb'))
# data_to_keep =[records[x] for x in users_to_keep]
# len(data_to_keep)
# #     del records
# diagnosis = [row[0] for row in data_to_keep]
# procedures = [row[1] for row in data_to_keep]
# medicines = [row[2] for row in data_to_keep]
# medicines_temp =[]
# diagnosis_temp = []
# procedure_temp = []
# diagnosis_df = []

# for row in range(0,len(medicines),1):
#     medicine = medicines[row]
#     if len(medicine.split(" "))>15:
#         medicines_temp.append(medicine)
# 
#         diagnosis_temp.append(diagnosis[row])
#         user_diagnosis = diagnosis[row].split(" ")
#         for diagnosi in user_diagnosis:
#             diagnosis_df.append((row,diagnosi))
#         procedure_temp.append(procedures[row])
#     counter+=1
# diagnosis_df = pd.DataFrame(diagnosis_df)
# diagnosis_df.columns= ['user','diagnosis']
# medicines = medicines_temp
# diagnosis= diagnosis_temp
# procedures = procedure_temp
# #     medicines_temp = []
# #     for medicine in medicines:
# #         if len(medicine.split(" "))>15:
# #             medicines_temp.append(medicine)
# #     medicines = medicines_temp
# medicines_tokenizer = Tokenizer()
# medicines_tokenizer.fit_on_texts(medicines)
# medicines_maxlen = max([len(line.split(' ')) for line in medicines])
# sequences = medicines_tokenizer.texts_to_sequences(medicines)

# padded_medicines = pad_sequences(sequences, maxlen=medicines_maxlen, padding='post', truncating='post')
# diagnosis_maxlen=max([len(line.split(' ')) for line in diagnosis])
# procedure_maxlen=max([len(line.split(' ')) for line in procedures])
# medicines_maxlen = max([len(line.split(' ')) for line in medicines])
# # Creating a bag-of-words for Diagnosis and Procedures sets
# # Each diagnosis/procedure code is taken from the EHR input file
# diagnosis_tokenizer = Tokenizer()
# diagnosis_tokenizer.fit_on_texts(diagnosis)
# sequences = diagnosis_tokenizer.texts_to_sequences(diagnosis)

# diagnosis_= pad_sequences(sequences, maxlen=diagnosis_maxlen, padding='post', truncating='post')

# procedure_tokenizer=Tokenizer()
# procedure_tokenizer.fit_on_texts(procedures)
# sequences=procedure_tokenizer.texts_to_sequences(procedures)

# procedure_=pad_sequences(sequences,maxlen=procedure_maxlen,padding='post',truncating='post')
# diseases_to_train = []
# medicines_to_train = []
# for x,z, y in zip(diagnosis_, procedure_,padded_medicines):
#     diseases_to_train.append([x,z])
#     medicines_to_train.append([y])
# diagnosis_icd =pd.read_csv('D_ICD_DIAGNOSES.csv')
# diagnosis_icd= diagnosis_icd[['ICD9_CODE','SHORT_TITLE']]
# procedures_icd =pd.read_csv('D_ICD_PROCEDURES.csv')
# procedures_icd= procedures_icd[['ICD9_CODE','SHORT_TITLE']]
atc_drug= pickle.load(open( "atc_drug.pkl", "rb" ))

print(f'Users : {len(train_df.user.unique())}, Drugs : {len(train_df.drug.unique())}')
print(train_df.shape)
patient_ids= list(train_df['user'].unique())
train_df.head()


We can view the correspondance between the drugs we have (codes from the dataset) and how we encode them in columns

In [ ]:
data_list = train_df.values.tolist()
overlapping_drugs = []
current_patient = data_list[0][0]
current_drugs =[]
for p_d in data_list:
    if p_d[0] == current_patient:
        current_drugs.append(p_d[1])
    else:
        overlapping_drugs.append(current_drugs)
        current_drugs = []
        current_drugs.append(p_d[1])
        current_patient = p_d[0]
drugs = []
dict_drug = {}
count= 0
for drug_list in overlapping_drugs:
    for drug in drug_list:
        if drug not in dict_drug.keys():
            dict_drug[drug] = count
            drugs.append(count)
            count+=1
drugs = np.asarray(drugs)
dict_drug['END'] = count
medicineSet=[]
new_medicines = []
for row in medicines:
    medicine_row = []
    for item in row.split(' '):
        medicine_row.append(item)
        if item not in medicineSet:
            medicineSet.append(item)
    new_medicines.append(medicine_row)
if dataset=='medical_2':
    medicineSet1=[]
    new_test_medicines = []
    for row in test_medicines:
        medicine_row = []
        for item in row.split(' '):
            medicine_row.append(item)
            if item not in medicineSet:
                medicineSet.append(item)
        new_test_medicines.append(medicine_row)
drug2id={drug:id for id,drug in enumerate(medicineSet)}
drug2id['END']=len(drug2id)
drug2id

We iterate each user and create an array with each one corresponding index to interests

In [ ]:
patient_drug_list = []
if dataset =='medical_2':
    patient_test_drug_list=[]
    for patient_drugs in new_test_medicines:

        drugs_per_patient = []
        for drug in patient_drugs: 
            drugs_per_patient.append(drug2id[drug])

        patient_test_drug_list.append(drugs_per_patient)

for patient_drugs in new_medicines:

    drugs_per_patient = []
    for drug in patient_drugs: 
        drugs_per_patient.append(drug2id[drug])

    patient_drug_list.append(drugs_per_patient)
patient_drug_list

In [ ]:
# medicines = []
# for i in range(0,len(patient_drug_list)):
#     user_medicine = f'{patient_drug_list[i][0]}'
#     for drug in range(1,len(patient_drug_list[i])):
#         user_medicine+= f' {patient_drug_list[i][drug]}'
#     medicines.append(user_medicine)
# medicines_maxlen = max([len(line.split(' ')) for line in medicines])
# medicines_tokenizer = Tokenizer()
# medicines_tokenizer.fit_on_texts(medicines)


We define the train split and split into train and test

In [ ]:
np.random.seed(seed)
train_set=[]
test_set =[]
count = 0
train_medicines = []

for user_interests in patient_drug_list:
    user_interests = np.asarray(user_interests)

    indices = np.arange(len(user_interests))

    np.random.shuffle(indices)

    train_end = int(train_test_split*len(user_interests))
    train_indices = indices[:train_end]

    train_indices = user_interests[train_indices]

    test_indices = indices[train_end:]
    test_indices = user_interests[test_indices]
    train_interests = np.zeros(len(drugs))
    test_interests = np.zeros(len(drugs))
    print(train_indices)
    train_interests[train_indices] = 1
    train_medicines.append(train_indices)
    train_set.append(train_interests)
    test_interests[test_indices] = 1
    test_set.append(test_interests)
    print(f'Patient {count} has taken {len(user_interests)} drugs, {len(train_indices)} drugs are in the train set, {len(test_indices)} drugs are in his test set')
    count+=1
cancer_data = 0
# rl_data_x, rl_data_y, drug2id = load_records( cancer_data =cancer_data,min_items= min_items, train_test_split = train_test_split,dataset = dataset)



We calcuate the user similarities and item similarities

In [ ]:
# if dataset =='mimic':
#     diagnosis_dict = {}# We create a dict in order to have each diagnosis as an int
#     count = 0
#     for i in list(diagnosis_df['diagnosis'].unique()):

#         diagnosis_dict[i] = count
#         count+=1
    
#     diagnosis_train_set = np.zeros(shape = [len(diagnosis_df['user'].unique()),len(diagnosis_df['diagnosis'].unique())])
#     #diagnosis_train_set is an array with rows as patients and columns as diseases
#     diagnosis_list = diagnosis_df.values.tolist()
#     for i in diagnosis_list:
#         diagnosis_train_set[i[0]][diagnosis_dict[i[1]]] = 1#We set ones to the diseases of each patient
#     diagnosis_train_set

In [ ]:
net = MatrixFactorization(
    np.asarray(train_set).shape[0], np.asarray(train_set).shape[1], 
    n_factors=5)

def user_user_cosine_similarity(train_set):
    A_sparse = sparse.csr_matrix(train_set)
    user_similarities = cosine_similarity(A_sparse)    
    return user_similarities


def drug_drug_cosine_similarity(train_set):
    A_sparse = sparse.csr_matrix(train_set)
    drug_similarities = cosine_similarity(A_sparse.T)    
    return drug_similarities

def mf_similarity(train_set):
    from scipy.sparse.linalg import svds
    A_sparse = sparse.csr_matrix(train_set)
    U, sigma, Vt = svds(A_sparse, k = 50)
    #drug_similarities = cosine_similarity(Vt.T)
    sigma = np.diag(sigma)
    user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) 
    return user_predicted_ratings
def mf_similarity_2(train_set,ddi=False,ddi_net = None):
    from scipy.sparse.linalg import svds
    if ddi:
        model = pycmf.CMF(n_components=50)
        U,V,Z =model.fit_transform(np.asarray(train_set),ddi_net)
        drug_similarities = cosine_similarity(V)
    else:
        A_sparse = sparse.csr_matrix(train_set)

        U, sigma, Vt = svds(A_sparse, k = 50)
        drug_similarities = cosine_similarity(Vt.T)
#     sigma = np.diag(sigma)
#     user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) 
    return drug_similarities

def pathsim_drug_drug_similarity(train_set,ddi=0,ddi_net = None):
    train_data = np.asarray(train_set)
    patients= []
    for i in range(0,train_data.shape[0]):
        patients.append(f'P{i}')
    drugs = []
    for i in range(0,train_data.shape[1]):
        drugs.append(f'D{i}')

    g = nx.Graph()
    g.add_nodes_from(patients, entity='P')
    g.add_nodes_from(drugs, entity='D')
    edges = np.where(train_data==1)
    for i in range(0,len(edges[0])):
        patient = 'P'+str(edges[0][i])
        drug = 'D'+str(edges[1][i])
        g.add_edge(patient,drug, edge_type='PD') 
    if ddi:
        for ddi_pair in ddi_net:
            d1 = 'D'+str(ddi_pair[0])
            d2 = 'D'+str(ddi_pair[1])
            g.add_edge(d1,d2,edge_type = 'DD')
    path_len = 1
    target_entity = 'D'
    adj_entity = 'P'
    G= g
    target_nodes = [node for node, attr in G.nodes(data=True) if attr['entity'] == target_entity]
    adj_nodes = [node for node, attr in G.nodes(data=True) if attr['entity'] == adj_entity]

    # ------ Calculating Adjacency matrix between target and adjacency nodes
    adj = defaultdict(list)

    for n1 in target_nodes:
        adj[n1] = defaultdict(list)
        for n2 in adj_nodes:
            adj[n1][n2] = len([path for path in nx.all_simple_paths(G, n1, n2, path_len)])

    # ------ Calculating similarities between target nodes
    sim = defaultdict(list)
    n_paths = defaultdict(list)

    for n1 in target_nodes:
        sim[n1] = defaultdict(int) #list
        n_paths[n1] = defaultdict(int)
        for n2 in target_nodes:
            sim[n1][n2] = -1
            n_paths[n1][n2] = 0

    for n1 in target_nodes:
        for n2 in target_nodes:
            if sim[n2][n1] == -1:
                n_connecting_paths = 0
                numerator = 0
                denominator = 0
                for n3 in adj_nodes:
                    n_connecting_paths += adj[n1][n3] * adj[n2][n3]
                    numerator += 2 * adj[n1][n3] * adj[n2][n3]
                    denominator += adj[n1][n3] ** 2 + adj[n2][n3] ** 2
                sim[n1][n2] = numerator / denominator if denominator != 0 else 0
                sim[n2][n1] = sim[n1][n2]
                n_paths[n1][n2] = n_connecting_paths
                n_paths[n2][n1] = n_paths[n1][n2]
    df = pd.DataFrame(data =sim)
    if ddi:
        target_entity = 'D'
        adj_entity = 'D'
        G= g
        target_nodes = [node for node, attr in G.nodes(data=True) if attr['entity'] == target_entity]
        adj_nodes = [node for node, attr in G.nodes(data=True) if attr['entity'] == adj_entity]

        # ------ Calculating Adjacency matrix between target and adjacency nodes
        adj = defaultdict(list)

        for n1 in target_nodes:
            adj[n1] = defaultdict(list)
            for n2 in adj_nodes:
                adj[n1][n2] = len([path for path in nx.all_simple_paths(G, n1, n2, path_len)])

        # ------ Calculating similarities between target nodes
        sim_ddi = defaultdict(list)
        n_paths = defaultdict(list)

        for n1 in target_nodes:
            sim_ddi[n1] = defaultdict(int) #list
            n_paths[n1] = defaultdict(int)
            for n2 in target_nodes:
                sim_ddi[n1][n2] = -1
                n_paths[n1][n2] = 0

        for n1 in target_nodes:
            for n2 in target_nodes:
                if sim_ddi[n2][n1] == -1:
                    n_connecting_paths = 0
                    numerator = 0
                    denominator = 0
                    for n3 in adj_nodes:
                        n_connecting_paths += adj[n1][n3] * adj[n2][n3]
                        numerator += 2 * adj[n1][n3] * adj[n2][n3]
                        denominator += adj[n1][n3] ** 2 + adj[n2][n3] ** 2
                    sim_ddi[n1][n2] = numerator / denominator if denominator != 0 else 0
                    sim_ddi[n2][n1] = sim_ddi[n1][n2]
                    n_paths[n1][n2] = n_connecting_paths
                    n_paths[n2][n1] = n_paths[n1][n2]
        df_ddi = pd.DataFrame(data = sim_ddi)
        
        
        df = df.add(-1*df_ddi)
    print(df)
    n_paths_matrix = n_paths
    return df.values


In [ ]:
user_similarities = user_user_cosine_similarity(train_set) 
# if dataset=='mimic':
#     user_disease_similarities = user_user_cosine_similarity(diagnosis_train_set)   
#     user_disease_similarities = (similarity_coefficient* user_similarities) + ((1-similarity_coefficient)* user_disease_similarities)
drug_similarities = drug_drug_cosine_similarity(train_set)             
if 'MF' in methods:      
    mf_similarities = mf_similarity(train_set)
    print(mf_similarities.shape)
if 'MF2' in methods:
    mf_2_similarities = mf_similarity_2(train_set)
    print(mf_2_similarities.shape,mf_2_similarities)
if 'Pathsim' in methods:
    pathsim_similarities = pathsim_drug_drug_similarity(train_set)
    random_pathsim = np.random.rand(pathsim_similarities.shape[0],pathsim_similarities.shape[1])

We define the neighborhood for the user based recommendation. We then calculate predictions based on the neighboor

In [ ]:
drug_rank = {value:key for key, value in drug2id.items()}
drug_rank

If we have an auxiliary network, like in Decagon or Medical, we can define an auxiliary similarity matrix

In [ ]:
ddi_similarities = np.ones(shape=drug_similarities.shape)
counter= 0
for i in range(0,df_ddi.shape[0]):
    drug_1 = df_ddi.iloc[i]['Drug1']
    drug_2 = df_ddi.iloc[i]['Drug2']
    if (drug_1 in dict_drug.keys()) and (drug_2 in dict_drug.keys()):
        counter+=1
        ddi_similarities[dict_drug[drug_1],dict_drug[drug_2]] = 0
        ddi_similarities[dict_drug[drug_2],dict_drug[drug_1]] = 0
print(counter)
ddi_similarities= (ddi_similarities-ddi_similarities.mean())/ddi_similarities.std()
min_p = ddi_similarities.min()
max_p = ddi_similarities.max()
ddi_similarities = (ddi_similarities-min_p)/(max_p-min_p)
pair_list = []
for pair in df_ddi[['Drug1','Drug2']].values:
    if (pair[0] in dict_drug.keys()) and (pair[1] in dict_drug.keys()):
        if [dict_drug[pair[0]],dict_drug[pair[1]]] not in pair_list:
            pair_list.append([dict_drug[pair[0]],dict_drug[pair[1]]])
if 'MF_ddi' in methods:
    mf_ddi_similarities = mf_similarity_2(train_set,ddi=1,ddi_net = ddi_similarities)
if 'Pathsim_ddi' in methods:
    pathsim_ddi_similarities = pathsim_drug_drug_similarity(train_set,ddi=1,ddi_net = pair_list)
    random_pathsim_ddi = np.random.rand(pathsim_ddi_similarities.shape[0],pathsim_ddi_similarities.shape[1])

<h2>PRECISION AND RECALL </h2>

Implement some helping functions

In [ ]:
def calculate_precision_recall(recs,test_set,user_id,results_text_name,train_set):
    if dataset =='mimic':
        with open(results_text_name, 'a') as f:


            if len(recs)==top_n:

                prescriptions = np.asarray(test_set)[user_id,:]
                train_prescriptions = np.asarray(train_set)[user_id,:]
                prescriptions = np.where(prescriptions==1)[0]
                train_prescriptions = np.where(train_prescriptions==1)[0]
                train_medicines = []

                if dataset =='mimic':
                    patient_id = patient_ids[user_id]
#                     print(patient_id, user_id, train_prescriptions)
                    train_ndc = []
                    for medicine in train_prescriptions:
                        train_ndc.append(drug_rank[medicine])
                        if drug_rank[medicine] in atc_drug.keys():
                            train_medicines.append(atc_drug[drug_rank[medicine]])
                        else:
                            train_medicines.append(drug_rank[medicine])
#                     current_diagnosi = diagnosis[user_id]
#                     current_procedure = procedures[user_id]
#         #             print(diagnosis_icd[diagnosis_icd['ICD9_CODE']==diagnosi]['SHORT_TITLE'].values[0])
#         #             print(current_diagnosi)
#                     diagnosi = []
#                     procedure = []
#                     for x in current_diagnosi.split(" "):
#                         if len(diagnosis_icd[diagnosis_icd['ICD9_CODE']==x])>0:
#                             diagnosi.append(diagnosis_icd[diagnosis_icd['ICD9_CODE']==x]['SHORT_TITLE'].values[0])

#                     for x in current_procedure.split(" "):
#                         if len(procedures_icd[procedures_icd['ICD9_CODE']==int(x)])>0:
#                             procedure.append(procedures_icd[procedures_icd['ICD9_CODE']==int(x)]['SHORT_TITLE'].values[0])
#                         else:
#                             procedure.append(x)


                    medicines_recommended = []
                    correct_predictions = 0
                    for rec in recs:
                        if rec in prescriptions:
                            if drug_rank[rec] in atc_drug.keys():
                                medicines_recommended.append(f'** {atc_drug[drug_rank[rec]]} **')
                            else:
                                medicines_recommended.append(f'** {drug_rank[rec]} **')
                            correct_predictions+=1
                        else:
                            if drug_rank[rec] in atc_drug.keys():
                                medicines_recommended.append(atc_drug[drug_rank[rec]])
                            else:
                                medicines_recommended.append(drug_rank[rec])



                #diagnosis


                
                drug_names = []
                for drug in prescriptions:
                    if drug_rank[drug] in atc_drug.keys():
                        drug_names.append(atc_drug[drug_rank[drug]])
                    else:
                        drug_names.append(drug_rank[drug])
                if method =='IBCF':
                    if correct_predictions>2:
                        print(f'Patient ID {user_id}',file=f)

                        if dataset=='medical':
                            print(f"Patient's Diseases : {list(carc_df[carc_df['patient']==patient_id]['carcinoma'].values)}",file=f)
        #                     print(f'Farmako {rec} find in prescriptions {prescriptions}')
#                         elif dataset=='mimic':
#                             print(f"Patient's Diseases : {diagnosi}/{procedure}",file=f)
                        print(f'Train Medicines : {train_medicines}',file=f)
                        print(f'Predicted Medication {medicines_recommended}',file=f)
                        print(f'Doctor’s actual prescribed Medication {drug_names}',file=f)    
                else:
                    print(f'Patient ID {user_id}',file=f)
                    if dataset=='medical':
                        print(f"Patient's Diseases : {list(carc_df[carc_df['patient']==patient_id]['carcinoma'].values)}",file=f)
    #                     print(f'Farmako {rec} find in prescriptions {prescriptions}')
#                     elif dataset=='mimic':
#                         print(f"Patient's Diseases : {diagnosi}/{procedure}",file=f)
                    print(f'Train Medicines : {train_medicines}',file=f)
                    print(f'Train Medicines NDC : {train_ndc}',file=f)
                    print(f'Predicted Medication {medicines_recommended}',file=f)
                    print(f'Doctor’s actual prescribed Medication {drug_names}',file=f)    

    correct_predictions = np.count_nonzero(np.asarray(test_set)[user_id][recs])
    test_interests = np.count_nonzero(np.asarray(test_set)[user_id])
    precision = correct_predictions/len(recs)
    #print(f'Correct Predictions : {correct_predictions} - Test Interests : {test_interests} - Recommendations : {len(recs)}')
    recall = correct_predictions/test_interests
    return precision,recall    
   
def calculate_adj_matrix(train_set):
    train_data = np.asarray(train_set)
    patients= []
    for i in range(0,train_data.shape[0]):
        patients.append(f'P{i}')
    drugs = []
    for i in range(0,train_data.shape[1]):
        drugs.append(f'D{i}')

    g = nx.Graph()
    g.add_nodes_from(patients, entity='P')
    g.add_nodes_from(drugs, entity='D')
    edges = np.where(train_data==1)
    for i in range(0,len(edges[0])):
        patient = 'P'+str(edges[0][i])
        drug = 'D'+str(edges[1][i])
        g.add_edge(patient,drug, edge_type='PD') 

    path_len = 1
    target_entity = 'D'
    adj_entity = 'P'
    G= g
    target_nodes = [node for node, attr in G.nodes(data=True) if attr['entity'] == target_entity]
    adj_nodes = [node for node, attr in G.nodes(data=True) if attr['entity'] == adj_entity]

    # ------ Calculating Adjacency matrix between target and adjacency nodes
    adj = defaultdict(list)

    for n1 in target_nodes:
        adj[n1] = defaultdict(list)
        for n2 in adj_nodes:
            adj[n1][n2] = len([path for path in nx.all_simple_paths(G, n1, n2, path_len)])
    return adj
def calculate_adj_matrix_ddi(train_set,ddi_net):
    train_data = np.asarray(train_set)
    patients= []
    for i in range(0,train_data.shape[0]):
        patients.append(f'P{i}')
    drugs = []
    for i in range(0,train_data.shape[1]):
        drugs.append(f'D{i}')
    
    g = nx.Graph()
    g.add_nodes_from(patients, entity='P')
    g.add_nodes_from(drugs, entity='D')
    edges = np.where(train_data==1)
    for i in range(0,len(edges[0])):
        patient = 'P'+str(edges[0][i])
        drug = 'D'+str(edges[1][i])
        g.add_edge(patient,drug, edge_type='PD') 
    for ddi_pair in ddi_net:
            d1 = 'D'+str(ddi_pair[0])
            d2 = 'D'+str(ddi_pair[1])
            g.add_edge(d1,d2,edge_type = 'DD')
    path_len = 1
    target_entity = 'D'
    adj_entity = 'D'
    G= g
    target_nodes = [node for node, attr in G.nodes(data=True) if attr['entity'] == target_entity]
    adj_nodes = [node for node, attr in G.nodes(data=True) if attr['entity'] == adj_entity]

    # ------ Calculating Adjacency matrix between target and adjacency nodes
    adj = defaultdict(list)

    for n1 in target_nodes:
        adj[n1] = defaultdict(list)
        for n2 in adj_nodes:
            adj[n1][n2] = len([path for path in nx.all_simple_paths(G, n1, n2, path_len)])
    return adj
def calculate_drugs_metapaths(drugs,adj,patient_dict):
    
    
    adj_list = []
    #patient_drugs = list(np.where(train_data[0]==1)[0])
    total_drug_metapaths= 0

    for current_drug in drugs:

        for p,number in adj[f'D{current_drug}'].items():
                if number ==1:
                    adj_list.append(p)

        for patient in adj_list:
            total_drug_metapaths += patient_dict[patient]
    return total_drug_metapaths


We will calculate the precision and recall for a range of patients

In [ ]:
adj_matrix = calculate_adj_matrix(train_set)
adj_ddi = calculate_adj_matrix_ddi(train_set,pair_list)
patient_dict ={}
for drug,patient in adj_matrix.items():
    for p,number in patient.items():
        
        if p in patient_dict.keys():
            1
        else:
            patient_dict[p]=0
        
        if number==1:
            patient_dict[p]+=1
drug_dict_ddi ={}
for drug,patient in adj_ddi.items():
    for p,number in patient.items():
        
        if p in drug_dict_ddi.keys():
            1
        else:
            drug_dict_ddi[p]=0
        
        if number==1:
            drug_dict_ddi[p]+=1
recommended_items = []
net.eval()
train_data =np.asarray(train_set)
precisions = []
recalls =[]
explainabilities = []
ddi_metric = []
results_text_name = 'results.txt'
for method in methods:
        with open(results_text_name, 'a') as f:
            print(method,file=f)
        method_precisions=[]
        method_recalls=[]
        method_ddi = []
        method_explainability = []
        print(f'Training started at {datetime.now()}')
        for recommendations in range (1,top_n+1):
            normal_ddi = 1
            temp_precision = []

            temp_recall = []
            temp_explainability = []

            print(f'Starting recommendations for method {method} for top - :{recommendations}')

            for user_id in range (0,np.asarray(train_set).shape[0]):
                total_metapaths = calculate_drugs_metapaths(drugs=list(np.where(np.asarray(train_set)[0]==1)[0]),adj=adj_matrix,patient_dict=patient_dict)
                
                if method =='IBCF':
                    recs = get_ib_recommendations(user_id, train_data,neighborhood,recommendations,drug_similarities)
                    if dataset in ddi_datasets:
                        for pair in combinations(recs,2):

                                if (list(pair) in pair_list):
                                    normal_ddi+=1
                elif method =='Pathsim':
                    recs = get_ib_recommendations(user_id, train_data,neighborhood,recommendations,0.8* pathsim_similarities + 0.2* random_pathsim)
                    if dataset in ddi_datasets:
                        for pair in combinations(recs,2):

                                if (list(pair) in pair_list):
                                    normal_ddi+=1
                elif method =='Pathsim_ddi':
                    recs = get_ib_recommendations(user_id, train_data,neighborhood,recommendations,0.8* pathsim_ddi_similarities + 0.2 * random_pathsim_ddi)
                    if dataset in ddi_datasets:
                        for pair in combinations(recs,2):

                                if (list(pair) in pair_list):
                                    normal_ddi+=1
                elif method == 'MF':
                    recs = get_mf_recommendations(user_id, train_data,neighborhood,recommendations,mf_similarities)
                    if dataset in ddi_datasets:
                        for pair in combinations(recs,2):

                            if (list(pair) in pair_list):
                                normal_ddi+=1
                elif method =='MF2':
                    recs = get_ib_recommendations(user_id, train_data,neighborhood,recommendations,mf_2_similarities)
                    if dataset in ddi_datasets:
                        for pair in combinations(recs,2):

                                if (list(pair) in pair_list):
                                    normal_ddi+=1
                elif method =='MF_ddi':
                    recs = get_ib_recommendations(user_id, train_data,neighborhood,recommendations,mf_ddi_similarities)
                    if dataset in ddi_datasets:
                        for pair in combinations(recs,2):

                                if (list(pair) in pair_list):
                                    normal_ddi+=1
                elif method == 'POP':
                    recs = get_pop_recommendations(user_id, train_data,neighborhood,recommendations,0)
                    if dataset in ddi_datasets:
                        for pair in combinations(recs,2):

                            if (list(pair) in pair_list):
                                normal_ddi+=1
                elif method == 'IBCF_ddi':
                    normalized_drug_similarities= (drug_similarities-drug_similarities.mean())/drug_similarities.std()
                    min_p = normalized_drug_similarities.min()
                    max_p = normalized_drug_similarities.max()
                    normalized_drug_similarities = (normalized_drug_similarities-min_p)/(max_p-min_p)
                    recs = get_ib_recommendations(user_id, train_data,neighborhood,recommendations,
                                                  drug_similarities+ddi_similarities)
                    if dataset in ddi_datasets:
                        for pair in combinations(recs,2):

                            if (list(pair) in pair_list):

                                normal_ddi+=1
                
                elif method =='random':
                    recs = get_random_recommendations(user_id,train_data,recommendations)
                    if dataset in ddi_datasets:
                        for pair in combinations(recs,2):

                            if (list(pair) in pair_list):
                                normal_ddi+=1


                else:
                        recs = get_nn_recommendations(user_id, train_data,neighborhood,recommendations,0,net=net)
                        if dataset in ddi_datasets:
                            for pair in combinations(recs,2):

                                if (list(pair) in pair_list):
                                    normal_ddi+=1
                recs_metapaths = calculate_drugs_metapaths(drugs=recs,adj=adj_matrix,patient_dict=patient_dict)
                if method=='Pathsim':
                    drug_drug_metapath = 0
                    for rec in recs:
                        drug_drug_metapath +=drug_dict_ddi[f'D{rec}']

                    recs_metapaths +=drug_drug_metapath
                explainability = 10* min(recs_metapaths,total_metapaths)/total_metapaths
                precision,recall = calculate_precision_recall(recs,test_set,user_id,results_text_name=results_text_name,train_set = train_set)
                temp_precision.append(precision)
                temp_recall.append(recall)
                temp_explainability.append(explainability)
            method_precisions.append(np.asarray(temp_precision).mean())
            method_recalls.append(np.asarray(temp_recall).mean())
            method_explainability.append(np.asarray(temp_explainability).mean())
            method_ddi.append(normal_ddi)
        ddi_metric.append(method_ddi)
        precisions.append(method_precisions)
        recalls.append(method_recalls)
        explainabilities.append(method_explainability)
        print(f'Training ended at {datetime.now()}')

        


Load DQN agent and produce results

In [ ]:
z = zip(precisions,recalls)

for precision,recall in z:
    plt.plot(recall,precision)


# plt.plot(ub_recalls,ub_precisions, ib_recalls,ib_precisions,pop_recalls,pop_precisions)


plt.xlabel("Recall")
plt.ylabel("Precision")
plt.legend(methods)
plt.show()

In [ ]:
counter = 0
for method in methods:
    print(f'Method : {method}\n Precision: {np.round(precisions[counter],4)} \n Recall :   {np.round(recalls[counter],4)} \n Explainability :    {np.round(explainabilities[counter],4)}')
    counter+=1

In [ ]:
ddi_metric

In [ ]:
chart_ddi = []
random_metric = ddi_metric[-1]
for method_ddi in range(0,len(ddi_metric)-1):
    ddi = ddi_metric[method_ddi]
    method_metric = []
    
    for counter in range(0,len(ddi)):
        
        method_metric.append((ddi[counter]-random_metric[counter])/random_metric[counter]*100)
    chart_ddi.append(method_metric)

ddi_metric

#chart_ddi.append(random_metric)

In [ ]:
chart_ddi

In [ ]:
# if 'IBCF_ddi' in methods:
for i in range(0,len(chart_ddi)):
    plt.plot(list(np.arange(1,top_n+1)),chart_ddi[i])
plt.xlabel("Top-n")
plt.ylabel("DDI Change% over  Random")
plt.legend(methods[:-1])
plt.show()

In [ ]:
users_for_recommendations = np.asarray(train_set).shape[0]
adv_ddi = []
for metric in ddi_metric:
    worst_ddi = []
    counter = 2
    for top_rec in metric[1:]:
        all_possible_adv = users_for_recommendations*(counter*(counter-1))/2
        worst_ddi.append(top_rec/all_possible_adv)
        counter+=1
    adv_ddi.append(worst_ddi)
adv_ddi

In [ ]:
for i in range(0,len(adv_ddi)):
    plt.plot(list(np.arange(2,top_n+1)),adv_ddi[i])
plt.xlabel("Top-n")
plt.ylabel("DDI Rate")
plt.legend(methods)
plt.show()

In [ ]:
counter = 0
for method in methods:
    print(f'Method : {method} has {adv_ddi[counter]} DDI Rate')
    counter+=1